# Download Landsat data

In [1]:
# pip install landsatxplore

username = "senv_team"

password= "ketchup&Mustard1"

output_dir = 'E:/Christina/land/tmp_dir/'



In [2]:
# pip install landsatxplore

import tarfile
import os
import json
from landsatxplore.api import API
from landsatxplore.earthexplorer import EarthExplorer

from skimage import io
io.use_plugin('pil')
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image
import cv2


# settings

In [9]:
places = [
    {"place":"sanfran", "lat":37.47, "long":-122.37},
    {"place":"tokyo", "lat":34.83, "long":140.18},
    {"place":"sea", "lat":47.31, "long":-141.19},
    {"place":"schwechat", "lat":48.04, "long":16.31}
]

# Initialize a new API instance and get an access key
api = API(username, password)

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_ot_c2_l1',
    latitude=48.04,
    longitude=16.31,
    start_date='2013-01-01',
    end_date='2016-10-01',
    max_cloud_cover=10
)    


print(f"{len(scenes)} scenes found.")

api.logout()

27 scenes found.


# print scene information

In [10]:

# Process the result
for scene in scenes:
    print(scene)
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    #fname = f"{scene['landsat_product_id']}.geojson"
    #with open(fname, "w") as f:
    #    json.dump(scene['spatial_coverage'].__geo_interface__, f)

{'cloud_cover': 4, 'entity_id': 'LC81900262016274LGN02', 'display_id': 'LC08_L1TP_190026_20160930_20200906_02_T1', 'ordering_id': 'None', 'landsat_product_id': 'LC08_L1TP_190026_20160930_20200906_02_T1', 'landsat_scene_id': 'LC81900262016274LGN02', 'acquisition_date': datetime.datetime(2016, 9, 30, 0, 0), 'collection_category': datetime.datetime(2023, 10, 1, 0, 0), 'collection_number': 2, 'wrs_path': 190, 'wrs_row': 26, 'nadir-off_nadir': 'NADIR', 'roll_angle': 0.0, 'date_product_generated': datetime.datetime(2020, 9, 6, 0, 0), 'land_cloud_cover': 4.09, 'scene_cloud_cover': 4.09, 'start_time': datetime.datetime(2016, 9, 30, 9, 45, 10, 618992), 'stop_time': datetime.datetime(2016, 9, 30, 9, 45, 42, 388989), 'station_id': 'LGN', 'day-night_indicator': 'DAY', 'ground_control_points_model': 779, 'ground_control_points_version': 5, 'geometric_rmse_model': 5.961, 'geometric_rmse_model_x': 4.063, 'geometric_rmse_model_y': 4.361, 'image_quality': 9, 'processing_software_version': 'LPGS_15.3.1c

# load data and extract the tar files

In [11]:
ee = EarthExplorer(username, password)

i_scene = 0

# scene["landsat_scene_id"]
for scene in scenes:
    try:
        print(scene["landsat_scene_id"])
        ee.download(scene["landsat_scene_id"], output_dir=output_dir)
        
        tar = tarfile.open(os.path.join(output_dir, scene["display_id"] + ".tar"))
        
        tmp_path = os.path.join(output_dir, scene["display_id"])
        tar.extractall(path=tmp_path)
        tar.close()
        
        stacked_image = None
        images = os.listdir(tmp_path)

        for image_name in images:

            print(image_name)

            if "B2" in image_name:
                print("b")
                img_path = os.path.join(tmp_path, image_name) 
                b = io.imread(img_path, plugin='tifffile')
                #print(b.max())
                b = np.float64(b)

            if "B3" in image_name:
                print("g")
                img_path = os.path.join(tmp_path, image_name) 
                g = io.imread(img_path, plugin='tifffile')
                #print(g.max())
                g = np.float64(g)


            if "B4" in image_name:
                print("r")
                img_path = os.path.join(tmp_path, image_name) 
                r = io.imread(img_path, plugin='tifffile')
                #print(r.max())
                r = np.float64(r)

        b *= (1.0/(65536/5) )
        g *= (1.0/(65536/5) )
        r *= (1.0/(65536/5) )   

        b = np.clip(b, a_min = 0, a_max = 1) 
        g = np.clip(g, a_min = 0, a_max = 1) 
        r = np.clip(r, a_min = 0, a_max = 1) 
        
        print(np.unique(b))
        print(np.unique(g))
        print(np.unique(r))

        rgb = np.dstack((r,g,b))

        rgb = cv2.normalize(rgb, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)
        im = Image.fromarray(rgb)
        im.save(os.path.join(tmp_path, scene["display_id"]+"_RGB.jpg"))

        # plt.imshow(rgb)
        
        i_scene += 1
        
        if i_scene > 10:
            break

    except Exception as E:
        print(E)

ee.logout()

LC81900262016274LGN02
Download failed with dataset id 1 of 2. Re-trying with the next one.


1.16GB [02:40, 7.76MB/s]


LC08_L1TP_190026_20160930_20200906_02_T1_ANG.txt
LC08_L1TP_190026_20160930_20200906_02_T1_B1.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B10.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B11.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B2.TIF
b
LC08_L1TP_190026_20160930_20200906_02_T1_B3.TIF
g
LC08_L1TP_190026_20160930_20200906_02_T1_B4.TIF
r
LC08_L1TP_190026_20160930_20200906_02_T1_B5.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B6.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B7.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B8.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_B9.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_MTL.json
LC08_L1TP_190026_20160930_20200906_02_T1_MTL.txt
LC08_L1TP_190026_20160930_20200906_02_T1_MTL.xml
LC08_L1TP_190026_20160930_20200906_02_T1_QA_PIXEL.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_QA_RADSAT.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_SAA.TIF
LC08_L1TP_190026_20160930_20200906_02_T1_stac.json
LC08_L1TP_190026_20160930_20200906_02_T1_SZA.TIF
LC08_L1TP

10


LC08_L1TP_190027_20160930_20200906_02_T1_ANG.txt
LC08_L1TP_190027_20160930_20200906_02_T1_B1.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B10.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B11.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B2.TIF
b
LC08_L1TP_190027_20160930_20200906_02_T1_B3.TIF
g
LC08_L1TP_190027_20160930_20200906_02_T1_B4.TIF
r
LC08_L1TP_190027_20160930_20200906_02_T1_B5.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B6.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B7.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B8.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_B9.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_MTL.json
LC08_L1TP_190027_20160930_20200906_02_T1_MTL.txt
LC08_L1TP_190027_20160930_20200906_02_T1_MTL.xml
LC08_L1TP_190027_20160930_20200906_02_T1_QA_PIXEL.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_QA_RADSAT.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_SAA.TIF
LC08_L1TP_190027_20160930_20200906_02_T1_stac.json
LC08_L1TP_190027_20160930_20200906_02_T1_SZA.TIF
LC08_L1TP

...

LC08_L1TP_190027_20150827_20200908_02_T1_ANG.txt
LC08_L1TP_190027_20150827_20200908_02_T1_B1.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B10.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B11.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B2.TIF
b
LC08_L1TP_190027_20150827_20200908_02_T1_B3.TIF
g
LC08_L1TP_190027_20150827_20200908_02_T1_B4.TIF
r
LC08_L1TP_190027_20150827_20200908_02_T1_B5.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B6.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B7.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B8.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_B9.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_MTL.json
LC08_L1TP_190027_20150827_20200908_02_T1_MTL.txt
LC08_L1TP_190027_20150827_20200908_02_T1_MTL.xml
LC08_L1TP_190027_20150827_20200908_02_T1_QA_PIXEL.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_QA_RADSAT.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_SAA.TIF
LC08_L1TP_190027_20150827_20200908_02_T1_stac.json
LC08_L1TP_190027_20150827_20200908_02_T1_SZA.TIF
LC08_L1TP

1.23GB [02:51, 7.69MB/s]


LC08_L1TP_190026_20150726_20200908_02_T1_ANG.txt
LC08_L1TP_190026_20150726_20200908_02_T1_B1.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B10.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B11.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B2.TIF
b
LC08_L1TP_190026_20150726_20200908_02_T1_B3.TIF
g
LC08_L1TP_190026_20150726_20200908_02_T1_B4.TIF
r
LC08_L1TP_190026_20150726_20200908_02_T1_B5.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B6.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B7.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B8.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_B9.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_MTL.json
LC08_L1TP_190026_20150726_20200908_02_T1_MTL.txt
LC08_L1TP_190026_20150726_20200908_02_T1_MTL.xml
LC08_L1TP_190026_20150726_20200908_02_T1_QA_PIXEL.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_QA_RADSAT.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_SAA.TIF
LC08_L1TP_190026_20150726_20200908_02_T1_stac.json
LC08_L1TP_190026_20150726_20200908_02_T1_SZA.TIF
LC08_L1TP

1.18GB [02:48, 7.51MB/s]


LC08_L1TP_189027_20150719_20200908_02_T1_ANG.txt
LC08_L1TP_189027_20150719_20200908_02_T1_B1.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B10.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B11.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B2.TIF
b
LC08_L1TP_189027_20150719_20200908_02_T1_B3.TIF
g
LC08_L1TP_189027_20150719_20200908_02_T1_B4.TIF
r
LC08_L1TP_189027_20150719_20200908_02_T1_B5.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B6.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B7.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B8.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_B9.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_MTL.json
LC08_L1TP_189027_20150719_20200908_02_T1_MTL.txt
LC08_L1TP_189027_20150719_20200908_02_T1_MTL.xml
LC08_L1TP_189027_20150719_20200908_02_T1_QA_PIXEL.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_QA_RADSAT.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_SAA.TIF
LC08_L1TP_189027_20150719_20200908_02_T1_stac.json
LC08_L1TP_189027_20150719_20200908_02_T1_SZA.TIF
LC08_L1TP

# stuff that didn't work

In [ ]:
if False:
    #!/usr/bin/python

    # https://urs.earthdata.nasa.gov/documentation/for_users/data_access/python

    import requests # get the requsts library from https://github.com/requests/requests



    # overriding requests.Session.rebuild_auth to mantain headers when redirected

    class SessionWithHeaderRedirection(requests.Session):

        AUTH_HOST = 'urs.earthdata.nasa.gov'

        def __init__(self, username, password):

            super().__init__()

            self.auth = (username, password)



       # Overrides from the library to keep headers when redirected to or from

       # the NASA auth host.

        def rebuild_auth(self, prepared_request, response):

            headers = prepared_request.headers

            url = prepared_request.url



            if 'Authorization' in headers:

                original_parsed = requests.utils.urlparse(response.request.url)

                redirect_parsed = requests.utils.urlparse(url)



                if (original_parsed.hostname != redirect_parsed.hostname) and redirect_parsed.hostname != self.AUTH_HOST and original_parsed.hostname != self.AUTH_HOST:
                    del headers['Authorization']



            return



    # create session with the user credentials that will be used to authenticate access to the data



    session = SessionWithHeaderRedirection(username, password)



    # the url of the file we wish to retrieve

    url = "https://n5eil01u.ecs.nsidc.org/MOST/MOD10A1.006/2016.12.31/MOD10A1.A2016366.h14v03.006.2017002110336.hdf.xml"
    url = "https://oceancolor.gsfc.nasa.gov/showimages/MODISA/IMAGES/CHL/L3/2023/0914/AQUA_MODIS.20230914.L3m.DAY.CHL.chlor_a.4km.NRT.nc.png"
    url = "https://oceancolor.gsfc.nasa.gov/showimages/MODISA/IMAGES/RRS/L3/2023/0925/AQUA_MODIS.20230925.L3m.DAY.RRS.Rrs_412.4km.NRT.nc.png"

    # extract the filename from the url to be used when saving the file


    import os
    if not os.path.isdir("tmp_dir"):
        os.makedirs("tmp_dir")
    filename = os.path.join("tmp_dir", url[url.rfind('/')+1:])



    try:

        # submit the request using the session

        response = session.get(url, stream=True)

        print(response.status_code)



        # raise an exception in case of http errors

        response.raise_for_status()  



        # save the file

        with open(filename, 'wb') as fd:

            for chunk in response.iter_content(chunk_size=1024*1024):

                print(chunk)

                fd.write(chunk)



    except requests.exceptions.HTTPError as e:

        # handle any errors here

        print(e)

In [ ]:
if False:
    from usgs import api


    def submit_where_query():

        # USGS uses numerical codes to identify queryable fields
        # To see which fields are queryable for a specific dataset,
        # send off a request to dataset-fields

        results = api.dataset_fields('LANDSAT_8_C1', 'EE')

        for field in results['data']:
            print(field)

        # WRS Path happens to have the field id 20514
        where = {
            20514: '043'
        }
        results = api.search('LANDSAT_8_C1', 'EE', where=where, start_date='2017-04-01', end_date='2017-05-01', max_results=10, extended=True)

        for scene in results['data']['results']:
            print(scene)

    submit_where_query()

In [ ]:
if False:
    from usgs import api

    # Set the EarthExplorer catalog
    node = 'EE'# this indicates earth explorer website

    # Set the Hyperion and Landsat 8 dataset
    hyperion_dataset = 'EO1_HYP_PUB'
    landsat8_dataset = 'LANDSAT_8'

    # Set the scene ids
    hyperion_scene_id = 'EO1H1820422014302110K2_SG1_01'
    landsat8_scene_id = 'LC80290462015135LGN00'

    # Submit requests to USGS servers
    api.metadata(hyperion_dataset, node, [hyperion_scene_id])
    api.metadata(landsat8_dataset, node, [landsat8_scene_id])